In [1]:
from keras.datasets import boston_housing
(train_data,train_targets),(test_data,test_targets)=boston_housing.load_data()

Using TensorFlow backend.
C:\Users\20170507\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\20170507\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\20170507\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\20170507\Anaconda3\lib\site-packages\tensorflow\python\framewo

In [2]:
#正規化
mean=train_data.mean(axis=0)
train_data -= mean
std=train_data.std(axis=0)
train_data /= std

test_data -= mean
test_data /= std


In [16]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Activation, Dropout
def build_model():
    model=Sequential()
    model.add(Dense(64, input_shape=(train_data.shape[1], ), activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1))

    model.compile(optimizer='rmsprop',loss='mse',metrics=['mae'])
    return model

In [12]:
import numpy as np

k = 4  #進行 4 折交叉驗證
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []

for i in range(k):
    print('processing fold #', i)  #準備驗證資料：資料來自 #k 區塊
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
    #準備訓練資料：資料來自 #k 以外的所有區塊
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)

    model = build_model() #建構 Keras 模型(已編譯)
    model.fit(partial_train_data, partial_train_targets,
              epochs=num_epochs, batch_size=1, verbose=0) #訓練該模型 (在 silent 靜音模式下, verbose = 0)
    
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)  #以驗證資料來評估模型
    all_scores.append(val_mae)

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3


In [5]:
"""import numpy as np
from sklearn.model_selection import StratifiedKFold

kf=StratifiedKFold(n_splits=4,random_state=1, shuffle=False)

for train_index, val_index in kf.split(train_data,train_targets):
    #print("TRAIN:", train_index, "TEST:", val_index)
    partical_train_data, val_data = train_data[train_index], train_data[val_index]
    partical_train_targets, val_targets = train_targets[train_index], train_targets[val_index]"""

C:\Users\20170507\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


ValueError: Supported target types are: ('binary', 'multiclass'). Got 'continuous' instead.

In [13]:
all_scores

[2.1765008, 2.5082157, 2.6932156, 2.341867]

In [14]:
np.mean(all_scores)

2.4299498

In [ ]:
num_epochs = 500
all_mae_histories = []
for i in range(k):
    print('processing fold #', i)
    
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)

    model = build_model()
    md = model.fit(partial_train_data, partial_train_targets,
                        validation_data=(val_data, val_targets),
                        epochs=num_epochs, batch_size=1, verbose=0)

processing fold # 0
processing fold # 1
processing fold # 2


In [ ]:
mae_history=md.history['val_mean_absolute_error']
all_mae_histories.append(mae_history)
#all_mae_histories=np.array(all_mae_histories)
#all_mae_histories.shape

In [ ]:
average_mae_history=[
    np.mean([x[i] for x in all_mae_histories]) for i in range(epochs)]

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(1,len(average_mae_history)+1),average_mae_history)
plt.xlabel('epochs')
plt.ylabel('validation mae')
plt.show()

In [ ]:
def smooth_curve(points, factor=0.9):
    smoothed_points=[]
    for point in points:
        if smoothed_points:
            previous=smoothed_points[-1]
            smootged_points.append(point*factor+previous*(1-factor))
        else:
            smoothed_points.append(point)
    return smoothed_points

smooth_mae_history=smooth_curve(average_mae_history[10:])

plt(range(1,len(smooth_mae_history)+1),smooth_mae_history)
plt.xlabel('epochs')
plt.ylabel('validation mae')
plt.show()

In [ ]:
model=build_model()
model.fit(train_data,train_targets,epochs=80, batch_size=16, verbose=0)
test_score,test_mae_score=model(test_data,test_targets)